In [1]:
import pandas as pd
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.cityofnewyork.us", None)

# Example authenticated client (needed for non-public datasets):
# client = Socrata(data.cityofnewyork.us,
#                  MyAppToken,
#                  userame="user@example.com",
#                  password="AFakePassword")

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("h9gi-nx95", limit=50000)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)
df = results_df[['crash_date','crash_time','borough','longitude','latitude','location','on_street_name',
                        'number_of_persons_killed','number_of_persons_injured']]
df = df.sort_values(by=['crash_date'],ascending = True)
df = df.dropna()
df = df.reset_index(drop=True)

In [2]:
# Tidy
def parts_of_day(x):
    if (x < 3):
        return 'Midnight'
    elif (x >= 3 and x <6):
        return 'Toward morning'
    elif (x >= 6 and x < 9):
        return 'Early Morning'
    elif (x >= 9 and x < 12):
        return 'Late Morning'
    elif (x >= 12 and x < 16):
        return 'Early Afternoon'
    elif (x >= 16 and x < 18):
        return 'Late Afternoon'
    elif (x >= 18 and x <= 21):
        return 'Evening'
    else:
        return 'Night'

df['crash_date'] = pd.to_datetime(df['crash_date'])
df['day_part'] = 0
df['total_injured'] = 0

for i in range(len(df)):
    df['day_part'][i] = parts_of_day(int(df['crash_time'][i].split(':')[0]))
    df['total_injured'][i] = df['number_of_persons_killed'][i] + df['number_of_persons_injured'][i]

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [3]:
df

,crash_date,crash_time,borough,longitude,latitude,location,on_street_name,number_of_persons_killed,number_of_persons_injured,day_part,total_injured
0,2016-04-16,14:20,BROOKLYN,-73.9862,40.586277,"{'latitude': '40.586277', 'longitude': '-73.98...",WEST 17 STREET,0,0,Early Afternoon,0
1,2019-05-21,22:50,BROOKLYN,-73.9831200,40.6975400,"{'latitude': '40.69754', 'longitude': '-73.983...",GOLD STREET,0,0,Night,0
2,2020-04-17,1:50,MANHATTAN,-73.99046,40.77161,"{'latitude': '40.77161', 'longitude': '-73.990...",11 AVENUE,0,1,Midnight,1
3,2020-05-02,17:30,QUEENS,-73.79473,40.67376,"{'latitude': '40.67376', 'longitude': '-73.794...",ROCKAWAY BOULEVARD,0,0,Late Afternoon,0
4,2020-05-10,0:51,MANHATTAN,-73.94463,40.84103,"{'latitude': '40.84103', 'longitude': '-73.944...",WEST 165 STREET,0,2,Midnight,2
...,...,...,...,...,...,...,...,...,...,...,...
18551,2021-07-19,2:30,BROOKLYN,-73.944336,40.703587,"{'latitude': '40.703587', 'longitude': '-73.94...",MANHATTAN AVENUE,0,1,Midnight,1
18552,2021-07-19,14:00,QUEENS,-73.76389,40.66546,"{'latitude': '40.66546', 'longitude': '-73.763...",181 STREET,0,0,Early Afternoon,0
18553,2021-07-19,8:24,BRONX,-73.90574,40.823334,"{'latitude': '40.823334', 'longitude': '-73.90...",EAST 163 STREET,0,0,Early Morning,0
18554,2021-07-19,14:35,MANHATTAN,-74.00612,40.723747,"{'latitude': '40.723747', 'longitude': '-74.00...",VARICK STREET,0,0,Early Afternoon,0


In [4]:
df.to_csv('MVC_Location.csv',encoding = 'utf-8',index = False)